1. Import and Install the required libraries


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip uninstall protobuf -y
!{sys.executable} -m pip uninstall mediapipe -y
!{sys.executable} -m pip uninstall tensorflow -y
!{sys.executable} -m pip uninstall opencv-python -y
!{sys.executable} -m pip uninstall opencv-python-headless -y
!{sys.executable} -m pip install notebook jupyterlab jupyter_server nbclient nbconvert nbformat ipywidgets --user
!{sys.executable} -m pip install opencv-python --user
!{sys.executable} -m pip install matplotlib --user
!{sys.executable} -m pip install protobuf==4.25.3 --user
!{sys.executable} -m pip install mediapipe --user
!{sys.executable} -m pip install tensorflow --user
!{sys.executable} -m pip install opencv-python-headless --user
!{sys.executable} -m pip install opencv-contrib-python --user
!{sys.executable} -m pip install ipython --user
!{sys.executable} -m pip install subprocess32 --user  
!{sys.executable} -m pip install threading --user  
!{sys.executable} -m pip install signal --user
!{sys.executable} -m pip install numpy --user    









In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
from IPython.display import display, Image, clear_output
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
import mediapipe as mp
import subprocess
import threading
import signal
import io
import ipywidgets as widgets
from PIL import Image
import asyncio


In [ ]:
mp_holistic = mp.solutions.holistic #Holistic Model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_face_mesh = mp.solutions.face_mesh

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    return image

In [ ]:
def draw_styled_landmarks(image, results):
#     # Draw face landmarks
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,
                                mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))

#     # Draw pose landmarks
    if results.pose_landmarks:
       mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))

#     # Draw left hand landmarks
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))

#     # Draw right hand landmarks
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=2))

In [ ]:
# Create an Image widget
image_widget = widgets.Image()
display(image_widget)

async def video_stream():
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Optimize resolution
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    cap.set(cv2.CAP_PROP_FPS, 30)  # Set FPS if supported by the camera

    loop = asyncio.get_event_loop()
    
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = await loop.run_in_executor(None, cap.read)  # Async frame read
            if not ret:
                break
            
            frame, results = mediapipe_detection(frame, holistic)  # Process frame with MediaPipe
            
            # Draw landmarks
            if results.face_landmarks:
                mp_drawing.draw_landmarks(frame, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,
                                          mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                                          mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
            if results.left_hand_landmarks:
                mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            if results.right_hand_landmarks:
                mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            
            # Compress and optimize encoding
            _, buffer = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 75])
            frame_bytes = buffer.tobytes()
            
            # Update image widget
            image_widget.value = frame_bytes

        cap.release()

# Run video stream in a separate thread to keep the notebook responsive
thread = threading.Thread(target=lambda: asyncio.run(video_stream()), daemon=True)
thread.start()


In [ ]:
# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

3. EXTRACT KEYPOINT VALUES

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)

In [ ]:
# Keypoint extraction
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
pose

In [ ]:
lh

In [ ]:
face

2. Keypoints using MP Holistic

In [ ]:
# # Keypoint extraction (simplified for brevity)
# def extract_keypoints(results):
#     pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
#     face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
#     lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
#     rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
#     return np.concatenate([pose, face, lh, rh])